In [2]:
import customtkinter as ctk
from tkinter import filedialog

# 外観モードとテーマの設定（任意）
ctk.set_appearance_mode("light")    # "light"または"dark"
ctk.set_default_color_theme("blue")   # カラーテーマ

# メインウィンドウの作成
app = ctk.CTk()
app.geometry("400x200")
app.title("ディレクトリ選択のデモ")

def select_directory():
    # ディレクトリ選択ダイアログを表示
    directory_path = filedialog.askdirectory(title="ディレクトリを選択してください")
    if directory_path:
        # 選択されたパスをラベルに表示
        label.configure(text=f"選択されたディレクトリ:\n{directory_path}")
    else:
        label.configure(text="ディレクトリが選択されませんでした")

# ボタンウィジェット：クリックでディレクトリ選択ダイアログを表示
select_button = ctk.CTkButton(app, text="ディレクトリを選択", command=select_directory)
select_button.pack(pady=20)

# 選択結果を表示するラベル
label = ctk.CTkLabel(app, text="まだディレクトリが選択されていません")
label.pack(pady=20)

# メインループ開始
app.mainloop()


In [1]:
import torch

# CUDA が利用可能かどうかを確認
print("CUDA is available:", torch.cuda.is_available())

# 利用可能な GPU の数を確認
print("GPU device count:", torch.cuda.device_count())

if torch.cuda.is_available():
    # 現在の CUDA デバイスのインデックスを取得
    current_device = torch.cuda.current_device()
    print("Current CUDA device index:", current_device)
    # 現在のデバイスの名前を取得
    print("Device name:", torch.cuda.get_device_name(current_device))


CUDA is available: True
GPU device count: 1
Current CUDA device index: 0
Device name: NVIDIA GeForce RTX 4060 Ti


In [2]:
import torch
from basicsr.archs.rrdbnet_arch import RRDBNet
from PIL import Image
import numpy as np
import math

def tile_inference(model, input_tensor, tile_size, tile_pad, scale):
    """
    入力テンソル（[1, C, H, W]）に対して、タイル処理を行いモデル推論を適用する関数
    Args:
      model: 推論に使用するモデル（GPU上に配置済み）
      input_tensor: 入力画像テンソル [1, C, H, W]
      tile_size: タイルサイズ（例: 256）
      tile_pad: タイル間のオーバーラップ幅（例: 10）
      scale: 拡大率（例: 4）
    Returns:
      出力テンソル [1, C, H*scale, W*scale]
    """
    _, C, H, W = input_tensor.shape
    output = torch.zeros(1, C, H * scale, W * scale, device=input_tensor.device)
    h_tiles = math.ceil(H / tile_size)
    w_tiles = math.ceil(W / tile_size)
    
    for i in range(h_tiles):
        for j in range(w_tiles):
            # タイルの元の領域（paddingなし）
            h_start = i * tile_size
            h_end = min(h_start + tile_size, H)
            w_start = j * tile_size
            w_end = min(w_start + tile_size, W)
            
            # padding 分を含めた領域（ただし画像境界を超えないように）
            h_start_pad = max(h_start - tile_pad, 0)
            h_end_pad = min(h_end + tile_pad, H)
            w_start_pad = max(w_start - tile_pad, 0)
            w_end_pad = min(w_end + tile_pad, W)
            
            # 対象タイルを抽出
            input_tile = input_tensor[:, :, h_start_pad:h_end_pad, w_start_pad:w_end_pad]
            
            # タイルに対して推論（with torch.no_grad() で勾配計算なし）
            with torch.no_grad():
                output_tile = model(input_tile)
            
            # 入力で追加した padding に対応する出力側のオフセット（scale 倍）
            h_valid_start = (h_start - h_start_pad) * scale
            h_valid_end = h_valid_start + (h_end - h_start) * scale
            w_valid_start = (w_start - w_start_pad) * scale
            w_valid_end = w_valid_start + (w_end - w_start) * scale
            
            valid_output = output_tile[:, :, h_valid_start:h_valid_end, w_valid_start:w_valid_end]
            
            # 出力画像内の配置位置（scale 倍）
            H_out_start = h_start * scale
            H_out_end = h_end * scale
            W_out_start = w_start * scale
            W_out_end = w_end * scale
            
            output[:, :, H_out_start:H_out_end, W_out_start:W_out_end] = valid_output

    return output

# 1. モデルの定義（RealESRGAN_x4plus用のRRDBNet）
scale = 16
model = RRDBNet(num_in_ch=3, num_out_ch=3, num_feat=64,
                num_block=23, num_grow_ch=32, scale=scale)

# 2. 学習済み .pth ファイルのロード
checkpoint = torch.load('./models/RealESRGAN_x4plus.pth', map_location='cuda')
if 'params_ema' in checkpoint:
    model.load_state_dict(checkpoint['params_ema'])
else:
    model.load_state_dict(checkpoint)

# 3. 推論モードに設定し、CUDA上かつ半精度で動作させる
model.eval()
model = model.cuda().half()

# 4. 入力画像の読み込みと前処理
input_image = Image.open('./assets/input.jpg').convert('RGB')
img_np = np.array(input_image).astype('float32') / 255.0
img_tensor = torch.from_numpy(img_np).permute(2, 0, 1).unsqueeze(0).cuda().half()

# 5. タイル処理を用いた推論
tile_size = 256  # タイルサイズ（適宜調整）
tile_pad = 10    # タイルのパディング幅（オーバーラップ）
with torch.no_grad():
    output_tensor = tile_inference(model, img_tensor, tile_size, tile_pad, scale)

# 6. 結果の後処理と保存
output_tensor = output_tensor.squeeze(0).float().cpu().clamp_(0, 1)
output_image = Image.fromarray((output_tensor.permute(1, 2, 0).numpy() * 255).astype('uint8'))
output_image.save('./assets/output_image.jpg')

print("超解像処理が完了しました。")


ModuleNotFoundError: No module named 'torchvision.transforms.functional_tensor'